In [11]:
import json

from prettytable import PrettyTable
from stringMatcher import levenshtein_ratio_and_distance
from time import time

table = PrettyTable(field_names=["UserID", "Rating", "Tags", "Timestamp"])

## Load The database into the memory...

In [27]:
data = json.load(open("Data_Sets/dataFinal.json", "rb"))
GIS = json.load(open("Data_Sets/dataFinal_GIS.json", "rb"))

## Load the Global Secondary index in the memory...

In [26]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic                  = time()
    
    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(queryString, KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the anime_id of the anime which is the best match to the queryString...
    bestMatch_AnimeTitle = list(GIS)[bestMatchRatio_index]
    bestMatch_AnimeId    = GIS[bestMatch_AnimeTitle]

    # Now we have anime_id which is the primary key in our main data-store...
    # Retrieve the details of the anime from the anime_id
    AnimeInfo            = data[str(bestMatch_AnimeId )]
    
    # Mark the search completion time...
    toc                  = time()

    # Add Movie Title to the response...
    AnimeInfo["title"] = bestMatch_AnimeTitle
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : AnimeInfo
    }
    
    return responseModel

## Write a function to make a request and visualize the data returned by the search function...

In [28]:
def request(AnimeToSearch=""):
    # Anime name should always be in a string...
    AnimeToSearch = str(AnimeToSearch)
    # Check if the movie name is valid...
    if(AnimeToSearch.replace(" ", "").strip() == ""):
        return f"Invalid Anime Name"
    
    # Make a request to fetch the anime info...
    response = getClosestMatch(queryString = AnimeToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
    # otherwise... 
    data_to_print = f"\
    Title          : {response['response body']['title']}\n\
    Genre          : {response['response body']['genre']}\n\
    Duration       : {response['response body']['duration']}\n\
    Rating         : {response['response body']['member_rating']}\n\
    Episodes       : {response['response body']['episodes']}\n\
    "
    print(data_to_print)

## Finally Make a request...

In [29]:
request(AnimeToSearch="Hell Boy")

    Title          : Cinderella Boy
    Genre          : Action, Adventure, Mystery, Comedy, Seinen
    Duration       : 
    Rating         : 
    Episodes       : 
    
